In [2]:
import pandas as pd
import numpy as np

In [3]:
workers_answers = pd.read_csv("Crowd_Frame riscaricato/data/result/task1/Dataframe/workers_answers.csv")
workers_questionnaire = pd.read_csv("Crowd_Frame riscaricato/data/result/task1/Dataframe/workers_questionnaire.csv")

In [4]:
absolute_frequencies = workers_questionnaire['question_answer_text'].value_counts()
relative_frequencies = absolute_frequencies / len(workers_questionnaire)

average_confidence = workers_answers.groupby('document_index')['Confidence_value'].mean()
average_time_elapsed = workers_answers.groupby('document_index')['time_elapsed'].mean()

In [5]:
#da verificare
total_units = len(workers_answers)
matching_units = workers_answers[workers_answers['Reliability_value'] == workers_answers['Reliability_value']].shape[0]

percent_agreement = (matching_units / total_units) * 100

96.49122807017544